In [2]:
# imports
import numpy as np
import pandas as pd

# A simple solution to the scheduling problem

Before each assignment we should start with updating the three variables below - **trained_people**, **busy_people**, and **next_date**.

https://trinket.io/python/b9684623be

In [45]:
#make a copy of data until now before it's too late
last_version = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/documents/trips_task_assignment/all_trips_simple.xlsx')
last_version.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/documents/trips_task_assignment/all_trips_simple copy 2.xlsx')

In [46]:
trained_people_B = ['Alice', 'Franz', 'Verjinia', 'Michael', 'Henrik', 'Henrike']
trained_people_H = ['Verjinia', 'Sarah', 'Alice', 'Franz', 'Michael', 'Philip']

permanently_unavailable = ['Henrik', 'Beatriz', 'Ecem', 'Rosie', 'Ana', 'Mirja', 'Henrike', 'Sarah', 'Franz']

In [47]:
df_busy = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/documents/trips_task_assignment/business.xlsx')

In [48]:
indx_op = 30

In [49]:
next_date = df_busy['date-city'][indx_op]
print(next_date)

2025-03-21-H


In [50]:
busy_people = permanently_unavailable + df_busy['not-available-people'][indx_op].split(', ')
busy_people

['Henrik',
 'Beatriz',
 'Ecem',
 'Rosie',
 'Ana',
 'Mirja',
 'Henrike',
 'Sarah',
 'Franz',
 'Michael',
 'Philip',
 'Franz',
 'Mathilde',
 'Johanna',
 'Rita']

In [51]:
#reading the data table with the stored information
df = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/documents/trips_task_assignment/all_trips_simple.xlsx')

#selecting only the last 15 trips
df_simple = df.drop(df.index[0:-15], axis=0).reset_index(drop = True)

In [52]:
#pool of available people
available_people = list(set(df_simple.columns[2:-1]) - set(busy_people))

In [53]:
#specifying the location team
if next_date[-1] == 'H':
    trained_people = trained_people_H
if next_date[-1] == 'B':
    trained_people = trained_people_B
    
#available trained people
trained_people = list(set(trained_people) - set(busy_people))

In [54]:
#who went to the last trip --> decreasing chance that is going to the next one
went_to_last_trip = df_simple.columns[df_simple.loc[len(df_simple)-1] == 1].tolist()

In [55]:
#determining the scores for each person
trips_per_person, weighted_scores = [], []
for col in df_simple.columns[2:-2]:
    trips_per_person.append(sum(df_simple[col]))
    if col in went_to_last_trip:
        weighted_scores.append(sum(df_simple[col])/len(df_simple) + 1) #adding one to make sure people don't go twice in a row
    else:
        weighted_scores.append(sum(df_simple[col])/len(df_simple))

#creating a table with scores and trips per person
training_score_df = pd.DataFrame({'person': df_simple.columns[2:-2], \
                          'trips_per_person':trips_per_person, \
                          'weighted_score':weighted_scores}).sort_values(by=['weighted_score', 'trips_per_person'], ignore_index = True)

In [56]:
#calculating the scores of the trained people
scores = []
for person in trained_people:
    scores.append(training_score_df['weighted_score'][training_score_df['person'] == person].item())

[0.26666666666666666, 0.2]

In [41]:
#chosing who is going to the next trip

#trained person
trained = []
for i in range(len(training_score_df)):
    if training_score_df.person[i] in available_people and training_score_df.person[i] in trained_people:
        trained.append(training_score_df.person[i])
        if len(trained) == 2:
            break
            
#untrained
untrained = []
for i in range(len(training_score_df)):       
    if training_score_df.person[i] in available_people and training_score_df.person[i] not in trained_people:
        untrained.append(training_score_df.person[i])
        if len(untrained) == 3:
            break

if len(trained) < 2:
    replacement_people = ['none', untrained.pop(2)]
else:
    replacement_people = [trained.pop(1), untrained.pop(2)]
people_next_trip = trained + untrained
people_next_trip

['Verjinia', 'Chrys', 'Zhengzheng']

In [42]:
#adding the new information to the existing datatable
add_to_df = []
for person in df_simple.columns[2:-2]:
    if person in people_next_trip:
        add_to_df.append(1)
    else:
        add_to_df.append(0)
add_to_df.insert(0, next_date)
add_to_df.insert(1, str(people_next_trip)[1:-2])
add_to_df.insert(len(add_to_df), str(replacement_people)[1:-2])
add_to_df.insert(len(add_to_df), 0)
df.loc[len(df.index)] = add_to_df

In [43]:
print('The people going on the trip ' + next_date + ' are ' + str(people_next_trip)[1:-1])
print('The replacement people are ' + str(replacement_people)[1:-1])

The people going on the trip 2025-03-21-H are 'Verjinia', 'Chrys', 'Zhengzheng'
The replacement people are 'Alice', 'James'


In [44]:
df.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/documents/trips_task_assignment/all_trips_simple.xlsx', index = False)

## Questions 
- How do we define trained? After how many trips?

In [1]:
#moving average for the last 15 trips 
#summing the scores and not only the ones for the trips
#weight estimated number of hours

Tasks {number of people needed, weight (requierd time investment)}
-	Prepare and filter sucrose			{1, 2}
-	Travel to destination				{3, 10}
-	Prepare prep room and help upon arrival	{1, 1}
-	Clean up after last experiments		{1, 1}
-   Slicing in Berlin {3, 4}

- new people introduced in HTM and then added to the list
- available people - assume everyone unless mentioned
- Prepare and filter sucrose - to keep standardized, only a few people 
- assignment of people to cleaning up
- make a future assignment for the next 10 trips, then check availability
- select more people (+1) as a possible replacement, if availability changes
- have core groups for BIE and HAM, trained poeple are location-specific
- (off-topic) professional looking name tags
- two lists - cleaning team and sucrose team
- moving average of 15 trips 
- to be a trained person., one needs introduction ot gas bottle handling
- backup solution --> transport of chunck, not so much pressure, easier 
- close to trained - Henrike, Sarah, Philip